In [ ]:
!pip install -r ./requirements.txt

In [ ]:
import numpy as np
from pyspark.sql import SparkSession

spark = (SparkSession 
         .builder
         .appName("Cloud computing ENSAE project")
         .master("local[5]")
         .getOrCreate()
        )
sc = spark.sparkContext

In [ ]:
def mult(x):
    y = np.array([2])
    return x*y
 
x = np.arange(10000)
distData = sc.parallelize(x)
 
results = distData.map(mult).collect()
 

In [ ]:
results

In [ ]:
spark.stop()